In [1]:
import copy
%load_ext autoreload
%autoreload 2
%env PYTORCH_ENABLE_MPS_FALLBACK=1
import os
os.chdir("/home/kubach/project_sync/star_analysis")
print(os.getcwd())

env: PYTORCH_ENABLE_MPS_FALLBACK=1
/home/kubach/project_sync/star_analysis


In [2]:
from star_analysis.model.types import ModelTypes
from star_analysis.runner.sdss_runner import SdssRunner
from star_analysis.data.augmentations import Augmentations
from star_analysis.runner.sdss_runner import SdssRunConfig, SdssModelConfig
from star_analysis.model.neural_networks.losses.types import LossType
from star_analysis.runner.run import Run

In [3]:
runner = SdssRunner(project_name="sdss-losses")

In [4]:
run_config0 = SdssRunConfig(
    model_config=SdssModelConfig(
        learning_rate=1e-4,
        batch_size=80,
        model_type=ModelTypes.UNET,
        loss_type=LossType.MSE
    ),
    augmentation=Augmentations.NONE,
    shuffle_train=True
)

run_config1 = copy.deepcopy(run_config0)
run_config1.model_config.loss_type = LossType.DICE

run_config2 = copy.deepcopy(run_config0)
run_config2.model_config.loss_type = LossType.DA_DICE

run_config3 = copy.deepcopy(run_config0)
run_config3.model_config.loss_type = LossType.FOCAL

run_config4 = copy.deepcopy(run_config0)
run_config4.model_config.loss_type = LossType.DA_FOCAL

configs = [run_config0, run_config1, run_config2, run_config3, run_config4]
runs = [Run(config) for config in configs]

for run in runs:
    runner.add_run(run)

In [ ]:
from star_analysis.runner.runner import TuningModes
from star_analysis.runner.run import TrainerConfig

results = runner.tune(
    mode=TuningModes.ITERATIVE,
    runs=runs,
    trainer_config=TrainerConfig(
        logger=runner.logger,
        max_epochs=10,
    )
)
run_results = zip(runs, results)

[INFO][2023-06-28 20:03:16,951] image_downloader.py:download_exact:62: Downloading 8162 6 0080
[INFO][2023-06-28 20:03:35,386] image_downloader.py:download_exact:62: Downloading 8162 6 0080
INFO: GPU available: True (cuda), used: True
[INFO][2023-06-28 20:03:53,908] rank_zero.py:_info:48: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
[INFO][2023-06-28 20:03:53,909] rank_zero.py:_info:48: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
[INFO][2023-06-28 20:03:53,910] rank_zero.py:_info:48: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
[INFO][2023-06-28 20:03:53,910] rank_zero.py:_info:48: HPU available: False, using: 0 HPUs
[INFO][2023-06-28 20:03:54,106] image_downloader.py:download_exact:62: Downloading 8162 6 0080
[INFO][2023-06-28 20:04:11,964] image_downloader.py:download_exact:62: Downloading 8162 6 0080
INFO: GPU available: True (cuda), used: True
[INFO][2023-06-28 20:04:30,

Run already built. Reusing existing build.
Found 947 data/label pairs
Found 1 data/label pairs


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[INFO][2023-06-28 20:06:40,740] cuda.py:set_nvidia_flags:58: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name         | Type    | Params
-----------------------------------------
0 | loss_fn      | MseLoss | 0     
1 | architecture | Unet    | 14.3 M
-----------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.339    Total estimated model params size (MB)
[INFO][2023-06-28 20:06:40,744] model_summary.py:summarize:82: 
  | Name         | Type    | Params
-----------------------------------------
0 | loss_fn      | MseLoss | 0     
1 | architecture | Unet    | 14.3 M
-----------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.339    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
results

In [ ]:
best_run, result_best = max(run_results, key=lambda x: x[1][f'{x[0].id}/test_f1'])
print(f"Best run, {best_run.name}, achieved {best_run[f'{best_run.id}/test_f1']} test_f1")

for i, (run, result) in enumerate(run_results):
    print(f"Run {run.id}, {run.name}, achieved {result[f'{run.id}/test_f1']} test_f1")

runner.save_model(best_run.model)

In [ ]:
print(best_run.config.model_config)